In [35]:
# Basic imports
import os
import sys
import time
import timeit
import random
import pickle
import re
from itertools import product
import warnings

# System path modification
sys.path.insert(0, '..')

# Data handling
import pandas as pd
import numpy as np

# Machine learning imports
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer, MissingIndicator
from sklearn.model_selection import (
    train_test_split, KFold, StratifiedKFold, GroupKFold, StratifiedGroupKFold, LeaveOneOut, cross_validate, cross_val_score
)
from sklearn.linear_model import (
    LinearRegression, Lasso, LassoCV, MultiTaskLasso, MultiTaskLassoCV,
    ElasticNet, ElasticNetCV, MultiTaskElasticNet, MultiTaskElasticNetCV
)
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

from sklearn.cross_decomposition import PLSRegression
from sklearn.inspection import permutation_importance

# Statistic imports 
from scipy.stats import ks_2samp
from scipy.special import kl_div
from scipy.cluster.hierarchy import linkage, leaves_list
from scipy.spatial.distance import pdist, squareform

# Specialized imputation and visualization packages
import miceforest as mf
import missingno as msno
#from missforest import MissForest
#import magic
from src.gain import *

# Custom modules
from src.train import *
from src.functions import *
from src.plots import *
from src.dataset import *
from src.multixgboost import *
from src.wrapper import *
from src.debug import *

# Visualizatiokn 
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning and machine learning specific 
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
import xgboost as xgb
import shap

from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.models import (
    GatedAdditiveTreeEnsembleConfig,
    DANetConfig,
    TabTransformerConfig,
    TabNetModelConfig,
)

# Ignore warnings
warnings.filterwarnings("ignore")

# Print CUDA availability for PyTorch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

from omegaconf import DictConfig
torch.serialization.safe_globals([DictConfig])

True
1


## Load data 

In [36]:
data = load_pickle_data_palettes()

results_pickle_folder = "../pickle/"

# Unpack data
df_X, df_y, df_all, df_FinalCombination = data["df_X"], data["df_y"], data["df_all"], data["df_FinalCombination"]
dict_select = data["dict_select"]

# Unpack colormaps
full_palette, gender_palette, dx_palette = data["colormaps"].values()

In [37]:
idx_train = list(df_X.isna().any(axis=1))
idx_test = list(~df_X.isna().any(axis=1))

set_intersect_rid = set(df_all[idx_train].RID).intersection(set(df_all[idx_test].RID))
intersect_rid_idx = df_all.RID.isin(set_intersect_rid)

for i, bool_test in enumerate(idx_test): 
    if intersect_rid_idx.iloc[i] & bool_test:
        idx_test[i] = False
        idx_train[i] = True

print(sum(idx_test))

print(df_all[idx_test].RID)

df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]] = df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]].astype("int", errors='ignore')

test_indices = [i for i, val in enumerate(idx_test) if val]

13
3609    2002
5631    4167
5662    4176
5780    4215
5950    4349
6069    4292
6077    4453
6085    4489
6224    4505
6400    4576
6429    4300
7021    2374
7192    4179
Name: RID, dtype: int64


# Leave-One-Complete-Out (LOCO-CV)

## All features

In [38]:
random_state=42

# Continuous Imputer List (list of tuples with unique strings and corresponding instances)
continuous_imputer_list = [
    ("KNNImputer_5", KNNImputer(n_neighbors=5)),
]

# Ordinal Imputer List (list of tuples with unique strings and corresponding instances)
ordinal_imputer_list = [
    ("KNNImputer", KNNImputer(n_neighbors=1)),
]

# Predictive Models List (list of tuples with unique strings and corresponding instances)
predictive_models_list = [
    ("LinearRegression", LinearRegression()),
    ("MultiTaskElasticNet", MultiTaskElasticNet()),
    ("MultiTaskElasticNet_tuned", MultiTaskElasticNet(**{'alpha': 0.1, 'l1_ratio': 0.1})),
    ("MultiTaskLasso", MultiTaskLasso()),
    ("MultiTaskLasso_tuned", MultiTaskLasso(**{'alpha': 0.001})),
    ("RandomForestRegressor", RandomForestRegressor()),
    ("XGBoostRegressor", XGBoostRegressor()),
    ("XGBoostRegressor_tuned", XGBoostRegressor(**{'colsample_bytree': 0.8776807051588262, 'learning_rate': 0.13329520360246094, 'max_depth': 8, 'min_child_weight': 4, 'subsample': 0.5924272277627636})),
    ("TabNetRegressor_default", TabNetModelWrapper(n_a=8, n_d=8)),
    ("TabNetRegressor_custom", TabNetModelWrapper(n_a=32, n_d=32)),
    ("PLSRegression_4_components", PLSRegression(n_components=4))
]

In [39]:
ordinal_features = ['APOE_epsilon2', 'APOE_epsilon3', 'APOE_epsilon4']
continuous_features = [col for col in df_X.columns if col not in ordinal_features]

# Prepare Tabular configurations (shared for all PyTorch models)
data_config = DataConfig(
    target=df_y.columns.tolist(),
    continuous_cols=continuous_features,
    categorical_cols=ordinal_features
)
trainer_config = TrainerConfig(
    batch_size=1024, max_epochs=10, auto_lr_find=False,
    early_stopping="valid_loss", early_stopping_mode="min", early_stopping_patience=5,
    checkpoints="valid_loss", load_best=True, progress_bar="nones",
)
optimizer_config = OptimizerConfig()
head_config = LinearHeadConfig(dropout=0.1).__dict__

predictive_models_list += [
    ("GatedAdditiveTreeEnsembleConfig_tab", 
    TabularModelWrapper(
        GatedAdditiveTreeEnsembleConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        gflu_stages=6,
        gflu_dropout=0.0,
        tree_depth=5,
        num_trees=20,
        chain_trees=False,
        share_head_weights=True), data_config, trainer_config, optimizer_config 
    )),
    ("DANetConfig_tab",
    TabularModelWrapper(
        DANetConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_layers=8,
        k=5,
        dropout_rate=0.1), data_config, trainer_config, optimizer_config
    )),
    ("TabTransformerConfig_tab",
        TabularModelWrapper(
        TabTransformerConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        embedding_initialization="kaiming_uniform",
        embedding_bias=False), data_config, trainer_config, optimizer_config
    )),
    ("TabNetModelConfig_tab",
        TabularModelWrapper(
        TabNetModelConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        n_independent=2,
        n_shared=2), data_config, trainer_config, optimizer_config
    )),
]

In [40]:
# Generate all combinations
combinations = list(product(continuous_imputer_list, ordinal_imputer_list, predictive_models_list))

# Display all combinations
for continuous_imputer, ordinal_imputer, model in combinations:
    print(f"Continuous Imputer: {continuous_imputer[0]}, Ordinal Imputer: {ordinal_imputer[0]}, Model: {model[0]}")

print(f"Combinations of preprocessing and models to test : {len(combinations)}")

Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer, Model: LinearRegression
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer, Model: MultiTaskElasticNet
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer, Model: MultiTaskElasticNet_tuned
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer, Model: MultiTaskLasso
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer, Model: MultiTaskLasso_tuned
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer, Model: RandomForestRegressor
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer, Model: XGBoostRegressor
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer, Model: XGBoostRegressor_tuned
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer, Model: TabNetRegressor_default
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer, Model: TabNetRegressor_custom
Continuous Imputer: KNNImputer_5, Ordinal Imputer: KNNImputer, Model: PLSRegression_4_compon

In [41]:
# Initialize HDF5 file
results_file = '../pickle/training_3_loonona_dict_results.pickle'

In [42]:
if os.path.exists(results_file): 
    with open(results_file, "rb") as input_file:
        all_dict_results = pickle.load(input_file)

else : 
    all_dict_results = []

In [43]:
if False : 
    params_comb = [{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'GatedAdditiveTreeEnsembleConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'DANetConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'TabTransformerConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'TabNetModelConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'GatedAdditiveTreeEnsembleConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'DANetConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabTransformerConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabNetModelConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]}]

    for params in params_comb:
        all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={"params": params})

In [44]:
all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={"fitting_time":None})

In [45]:
for continuous_imputer, ordinal_imputer, model in combinations:
    name_continuous_imputer, continuous_imputer_instance = continuous_imputer
    name_ordinal_imputer, ordinal_imputer_instance = ordinal_imputer
    name_model, model_instance = model

    params = {
            "ordinal_imputer": name_ordinal_imputer, 
            "continuous_imputer": name_continuous_imputer, 
            "model": name_model, "train_shape" : [df_X.shape[0]-1, df_X.shape[1]],
            "test_shape": [1, df_X.shape[1]]
        }
    
    # Define the subset of keys you care about
    keys_to_check = ['ordinal_imputer', 'continuous_imputer', 'model']  # or whatever subset you want

    # Check if a result in all_dict_results has the same values for just those keys
    if any(all(result['params'].get(k) == params.get(k) for k in keys_to_check) for result in all_dict_results):
        print(f"Skipping existing combination (subset match): {[params[k] for k in keys_to_check]}")
        continue

    dict_results = {
            "params": params, 
            "imputation_time": [],
            "fitting_time": [], 
            "results_adj": [], 
            "results_org": []
        }

    for test_nloc in test_indices: 
        print(test_nloc)

        idx_train = [True for i in range(df_X.shape[0])]
        idx_test = [False for i in range(df_X.shape[0])]

        idx_test[test_nloc] = True
        idx_train[test_nloc] = False

        df_X_train = df_X.loc[idx_train]
        df_X_test = df_X.loc[idx_test]

        df_y_train = df_y.loc[idx_train]
        df_y_test = df_y.loc[idx_test]

        c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
        c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

        try: 
        
            # Now you can call your `train_model` function with these components
            fold_dict_results = train_imputer_model(
                df_X_train, df_X_test, df_y_train, df_y_test,
                c_train, c_test,
                ordinal_imputer_instance, name_ordinal_imputer,
                continuous_imputer_instance, name_continuous_imputer,
                model_instance, name_model,
                separate_imputers=True  # Or however you want to specify
            )
            
            dict_results["imputation_time"].append(fold_dict_results["imputation_time"]) 
            dict_results["fitting_time"].append(fold_dict_results["fitting_time"])  
            dict_results["results_adj"].append(fold_dict_results["results_adj"])  
            dict_results["results_org"].append(fold_dict_results["results_org"])  

        except Exception as e:  

            print(e)
            
    # Optionally keep the all_dict_results list updated
    all_dict_results.append(dict_results)

    # Save the updated results back to the pickle file
    with open(results_file, 'wb') as f:
        pickle.dump(all_dict_results, f)


Skipping existing combination (subset match): ['KNNImputer', 'KNNImputer_5', 'LinearRegression']
Skipping existing combination (subset match): ['KNNImputer', 'KNNImputer_5', 'MultiTaskElasticNet']
Skipping existing combination (subset match): ['KNNImputer', 'KNNImputer_5', 'MultiTaskElasticNet_tuned']
Skipping existing combination (subset match): ['KNNImputer', 'KNNImputer_5', 'MultiTaskLasso']
Skipping existing combination (subset match): ['KNNImputer', 'KNNImputer_5', 'MultiTaskLasso_tuned']
Skipping existing combination (subset match): ['KNNImputer', 'KNNImputer_5', 'RandomForestRegressor']
Skipping existing combination (subset match): ['KNNImputer', 'KNNImputer_5', 'XGBoostRegressor']
Skipping existing combination (subset match): ['KNNImputer', 'KNNImputer_5', 'XGBoostRegressor_tuned']
Skipping existing combination (subset match): ['KNNImputer', 'KNNImputer_5', 'TabNetRegressor_default']
Skipping existing combination (subset match): ['KNNImputer', 'KNNImputer_5', 'TabNetRegressor_c

In [46]:
# Store data (serialize)
with open(results_file, 'wb') as handle:
    pickle.dump(all_dict_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [47]:
results_file

'../pickle/training_3_loonona_dict_results.pickle'

In [48]:
with open(results_file, "rb") as input_file:
    dict_results_loo_nona = pickle.load(input_file)

In [49]:
df_results_nona = pd.json_normalize(dict_results_loo_nona)

# Train only on MRI features

In [50]:
idx_train = list(df_X.isna().any(axis=1))
idx_test = list(~df_X.isna().any(axis=1))

set_intersect_rid = set(df_all[idx_train].RID).intersection(set(df_all[idx_test].RID))
intersect_rid_idx = df_all.RID.isin(set_intersect_rid)

for i, bool_test in enumerate(idx_test): 
    if intersect_rid_idx.iloc[i] & bool_test:
        idx_test[i] = False
        idx_train[i] = True

In [51]:
df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]] = df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]].astype("int", errors='ignore')

In [52]:
df_X_train = df_X[dict_select["MRIth"]].loc[idx_train]
df_X_test = df_X[dict_select["MRIth"]].loc[idx_test]

df_y_train = df_y.loc[idx_train]
df_y_test = df_y.loc[idx_test]

c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

In [53]:
random_state=42
n_imputation_iter = 10

# Continuous Imputer List (list of tuples with unique strings and corresponding instances)
continuous_imputer_list = [
    ("NoImputer", KNNImputer(n_neighbors=1)),

]

# Ordinal Imputer List (list of tuples with unique strings and corresponding instances)
ordinal_imputer_list = [
    ("NoImputer", SimpleImputer(strategy="most_frequent")),
]

# Predictive Models List (list of tuples with unique strings and corresponding instances)
predictive_models_list = [
    ("LinearRegression", LinearRegression()),
    ("MultiTaskElasticNet", MultiTaskElasticNet()),
    ("MultiTaskElasticNet_tuned", MultiTaskElasticNet(**{'alpha': 0.1, 'l1_ratio': 0.1})),
    ("MultiTaskLasso", MultiTaskLasso()),
    ("MultiTaskLasso_tuned", MultiTaskLasso(**{'alpha': 0.001})),
    ("RandomForestRegressor", RandomForestRegressor()),
    ("XGBoostRegressor", XGBoostRegressor()),
    ("XGBoostRegressor_tuned", XGBoostRegressor(**{'colsample_bytree': 0.8776807051588262, 'learning_rate': 0.13329520360246094, 'max_depth': 8, 'min_child_weight': 4, 'subsample': 0.5924272277627636})),
    ("TabNetRegressor_default", TabNetModelWrapper(n_a=8, n_d=8)),
    ("TabNetRegressor_custom", TabNetModelWrapper(n_a=32, n_d=32)),
    ("PLSRegression_4_components", PLSRegression(n_components=4))
]

In [54]:
ordinal_features = ['APOE_epsilon2', 'APOE_epsilon3', 'APOE_epsilon4']
continuous_features = [col for col in df_X_train.columns if col not in ordinal_features]

# Prepare Tabular configurations (shared for all PyTorch models)
data_config = DataConfig(
    target=df_y_train.columns.tolist(),
    continuous_cols=dict_select["MRIth"],
    categorical_cols=[]
)
trainer_config = TrainerConfig(
    batch_size=1024, max_epochs=10, auto_lr_find=True,
    early_stopping="valid_loss", early_stopping_mode="min", early_stopping_patience=5,
    checkpoints="valid_loss", load_best=True, progress_bar="nones",
)
optimizer_config = OptimizerConfig()
head_config = LinearHeadConfig(dropout=0.1).__dict__
predictive_models_list += [
    ("GatedAdditiveTreeEnsembleConfig_tab", 
    TabularModelWrapper(
        GatedAdditiveTreeEnsembleConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        gflu_stages=6,
        gflu_dropout=0.0,
        tree_depth=5,
        num_trees=20,
        chain_trees=False,
        share_head_weights=True), data_config, trainer_config, optimizer_config 
    )),
    ("DANetConfig_tab",
    TabularModelWrapper(
        DANetConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_layers=8,
        k=5,
        dropout_rate=0.1), data_config, trainer_config, optimizer_config
    )),
    ("TabTransformerConfig_tab",
        TabularModelWrapper(
        TabTransformerConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        embedding_initialization="kaiming_uniform",
        embedding_bias=False), data_config, trainer_config, optimizer_config
    )),
    ("TabNetModelConfig_tab",
        TabularModelWrapper(
        TabNetModelConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        n_independent=2,
        n_shared=2), data_config, trainer_config, optimizer_config
    )),
]


In [55]:
# Generate all combinations
combinations = list(product(continuous_imputer_list, ordinal_imputer_list, predictive_models_list))

# Display all combinations
for continuous_imputer, ordinal_imputer, model in combinations:
    print(f"Continuous Imputer: {continuous_imputer[0]}, Ordinal Imputer: {ordinal_imputer[0]}, Model: {model[0]}")

print(f"Combinations of preprocessing and models to test : {len(combinations)}")

Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: LinearRegression
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskElasticNet
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskElasticNet_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskLasso
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskLasso_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: RandomForestRegressor
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: XGBoostRegressor
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: XGBoostRegressor_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: TabNetRegressor_default
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: TabNetRegressor_custom
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: PLSRegression_4_components
Continuous Imputer: NoImputer, Ordinal 

In [56]:
# Initialize HDF5 file
results_file = '../pickle/training_3_loonona_dict_results.pickle'

with open('../pickle/training_3_loonona_dict_results.pickle', "rb") as input_file:
    all_dict_results = pickle.load(input_file)

In [57]:
all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={"fitting_time":None})

In [58]:
for res in all_dict_results: 
    print(res["params"])

{'ordinal_imputer': 'KNNImputer', 'continuous_imputer': 'KNNImputer_5', 'model': 'LinearRegression', 'train_shape': [2893, 256], 'test_shape': [1, 256]}
{'ordinal_imputer': 'KNNImputer', 'continuous_imputer': 'KNNImputer_5', 'model': 'MultiTaskElasticNet', 'train_shape': [2893, 256], 'test_shape': [1, 256]}
{'ordinal_imputer': 'KNNImputer', 'continuous_imputer': 'KNNImputer_5', 'model': 'MultiTaskElasticNet_tuned', 'train_shape': [2893, 256], 'test_shape': [1, 256]}
{'ordinal_imputer': 'KNNImputer', 'continuous_imputer': 'KNNImputer_5', 'model': 'MultiTaskLasso', 'train_shape': [2893, 256], 'test_shape': [1, 256]}
{'ordinal_imputer': 'KNNImputer', 'continuous_imputer': 'KNNImputer_5', 'model': 'MultiTaskLasso_tuned', 'train_shape': [2893, 256], 'test_shape': [1, 256]}
{'ordinal_imputer': 'KNNImputer', 'continuous_imputer': 'KNNImputer_5', 'model': 'RandomForestRegressor', 'train_shape': [2893, 256], 'test_shape': [1, 256]}
{'ordinal_imputer': 'KNNImputer', 'continuous_imputer': 'KNNImp

In [59]:
from src.debug import *

rm_combinations = [{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'LinearRegression', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'MultiTaskElasticNet', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'MultiTaskLasso', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'RandomForestRegressor', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'XGBoostRegressor', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'XGBoostRegressor_tuned', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabNetRegressor_default', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabNetRegressor_custom', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'PLSRegression_4_components', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'MultiTaskElasticNet_tuned', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'MultiTaskLasso_tuned', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'GatedAdditiveTreeEnsembleConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'DANetConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabTransformerConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabNetModelConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]}, 
]
if False : 
    for par in rm_combinations:
        all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={"params":par})

In [60]:
for continuous_imputer, ordinal_imputer, model in combinations:
    name_continuous_imputer, continuous_imputer_instance = continuous_imputer
    name_ordinal_imputer, ordinal_imputer_instance = ordinal_imputer
    name_model, model_instance = model

    params = {
            "ordinal_imputer": name_ordinal_imputer, 
            "continuous_imputer": name_continuous_imputer, 
            "model": name_model, "train_shape" : [df_X.shape[0]-1, df_X.shape[1]],
            "test_shape": [1, df_X.shape[1]]
        }
    
    # Define the subset of keys you care about
    keys_to_check = ['ordinal_imputer', 'continuous_imputer', 'model']  # or whatever subset you want

    # Check if a result in all_dict_results has the same values for just those keys
    if any(all(result['params'].get(k) == params.get(k) for k in keys_to_check) for result in all_dict_results):
        print(f"Skipping existing combination (subset match): {[params[k] for k in keys_to_check]}")
        continue

    dict_results = {
            "params": params, 
            "imputation_time": [],
            "fitting_time": [], 
            "results_adj": [], 
            "results_org": []
        }

    for test_nloc in test_indices: 
        print(test_nloc)

        idx_train = [True for i in range(df_X.shape[0])]
        idx_test = [False for i in range(df_X.shape[0])]

        idx_test[test_nloc] = True
        idx_train[test_nloc] = False

        df_X_train = df_X[dict_select["MRIth"]].loc[idx_train]
        df_X_test = df_X[dict_select["MRIth"]].loc[idx_test]

        df_y_train = df_y.loc[idx_train]
        df_y_test = df_y.loc[idx_test]

        c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
        c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

        try: 
        
            # Now you can call your `train_model` function with these components
            fold_dict_results = train_imputer_model(
                df_X_train, df_X_test, df_y_train, df_y_test,
                c_train, c_test,
                ordinal_imputer_instance, name_ordinal_imputer,
                continuous_imputer_instance, name_continuous_imputer,
                model_instance, name_model,
                separate_imputers=True  # Or however you want to specify
            )
            
            dict_results["imputation_time"].append(fold_dict_results["imputation_time"]) 
            dict_results["fitting_time"].append(fold_dict_results["fitting_time"])  
            dict_results["results_adj"].append(fold_dict_results["results_adj"])  
            dict_results["results_org"].append(fold_dict_results["results_org"])  

        except Exception as e:  

            print(e)
            
    # Optionally keep the all_dict_results list updated
    all_dict_results.append(dict_results)

    # Save the updated results back to the pickle file
    with open(results_file, 'wb') as f:
        pickle.dump(all_dict_results, f)


Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'LinearRegression']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'MultiTaskElasticNet']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'MultiTaskElasticNet_tuned']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'MultiTaskLasso']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'MultiTaskLasso_tuned']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'RandomForestRegressor']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'XGBoostRegressor']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'XGBoostRegressor_tuned']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'TabNetRegressor_default']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'TabNetRegressor_custom']
Skipping existing combination (s

In [61]:
# Store data (serialize)
with open(results_file, 'wb') as handle:
    pickle.dump(all_dict_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [62]:
with open(results_file, "rb") as input_file:
    all_dict_results = pickle.load(input_file)

## Print Table for paper

In [63]:
with open('../pickle/training_3_loonona_dict_results.pickle', "rb") as input_file:
    dict_results_loo_nona = pickle.load(input_file)

In [64]:
def generate_metric_table(
    results_list,
    targets,
    metric_name="r2",
    source="Adjusted",
    csv_filename=None,
    sort_order="descending"
):
    from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, explained_variance_score
    from scipy.stats import pearsonr
    import pandas as pd
    import numpy as np

    # Choose the key
    key = "results_adj" if source.lower() == "adjusted" else "results_org"
    df_rows = []

    for res in results_list:
        result_blocks = res.get(key, [])
        if not isinstance(result_blocks, list) or len(result_blocks) == 0:
            continue

        # Aggregate all predictions across folds
        y_preds_all = {target: [] for target in targets}
        y_tests_all = {target: [] for target in targets}

        for fold in result_blocks:
            y_pred = fold["y_pred"]
            y_test = fold["y_test"]

            if y_pred.shape != y_test.shape:
                continue

            for i, target in enumerate(targets):
                if y_pred.shape[1] <= i:
                    continue
                y_preds_all[target].append(y_pred[0, i])
                y_tests_all[target].append(y_test[0, i])

        # Compute metric for each target
        target_metrics = []
        for target in targets:
            y_preds = y_preds_all[target]
            y_tests = y_tests_all[target]

            if len(y_preds) < 2:
                metric = float("nan")
            else:
                if metric_name == "r2":
                    metric = r2_score(y_tests, y_preds)
                elif metric_name == "mae":
                    metric = mean_absolute_error(y_tests, y_preds)
                elif metric_name == "mse":
                    metric = mean_squared_error(y_tests, y_preds)
                elif metric_name == "explained_variance":
                    metric = explained_variance_score(y_tests, y_preds)
                elif metric_name == "corr":
                    metric = pearsonr(y_tests, y_preds)[0]
                else:
                    raise ValueError(f"Unsupported metric: {metric_name}")

            target_metrics.append(metric)

        # Compute mean ± std across targets
        values = np.array(target_metrics, dtype=np.float64)
        mean_val = np.nanmean(values)
        std_val = np.nanstd(values)

        # Get times
        imp_times = np.array(res.get("imputation_time", []), dtype=np.float64)
        fit_times = np.array(res.get("fitting_time", []), dtype=np.float64)

        row = {
            "continuous_imputer": res["params"].get("continuous_imputer"),
            "ordinal_imputer": res["params"].get("ordinal_imputer"),
            "model": res["params"].get("model"),
            **{target: val for target, val in zip(targets, target_metrics)},
            "mean ± std": f"{mean_val:.3f} ± {std_val:.3f}",
            "imp_time (s)": f"{np.nanmean(imp_times):.1f} ± {np.nanstd(imp_times):.1f}",
            "fit_time (s)": f"{np.nanmean(fit_times):.1f} ± {np.nanstd(fit_times):.1f}",
            "_sort_val": mean_val  # hidden sort key
        }
        df_rows.append(row)

    df = pd.DataFrame(df_rows)

    # Sort by mean value
    df = df.sort_values(by="_sort_val", ascending=(sort_order == "ascending")).drop(columns="_sort_val")

    # Optional: Save to CSV
    if csv_filename:
        df.to_csv(csv_filename, index=False)

    # Format for LaTeX output
    latex_df = df.copy()
    for target in targets:
        latex_df[target] = latex_df[target].apply(lambda x: f"{x:.3f}" if pd.notnull(x) else "–")

    latex_output = latex_df.to_latex(index=False, escape=False)

    return df, latex_output


In [65]:
latex_df, latex_corr = generate_metric_table(
    results_list=dict_results_loo_nona,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='corr',
    source="Adjusted",
    csv_filename="../tables/3_training_loonona_corr_adjusted_sorted.csv",
    sort_order="descending"
)
print(latex_corr)

\begin{tabular}{llllllllll}
\toprule
continuous_imputer & ordinal_imputer & model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & mean ± std & imp_time (s) & fit_time (s) \\
\midrule
KNNImputer_5 & KNNImputer & RandomForestRegressor & 0.678 & 0.738 & 0.358 & 0.700 & 0.619 ± 0.152 & 2.5 ± 0.0 & 39.8 ± 0.5 \\
KNNImputer_5 & KNNImputer & TabNetRegressor_default & 0.678 & 0.556 & 0.489 & 0.711 & 0.609 ± 0.090 & 2.5 ± 0.0 & 12.4 ± 0.7 \\
KNNImputer_5 & KNNImputer & XGBoostRegressor & 0.620 & 0.685 & 0.272 & 0.680 & 0.564 ± 0.171 & 2.5 ± 0.0 & 1.1 ± 0.1 \\
KNNImputer_5 & KNNImputer & GatedAdditiveTreeEnsembleConfig_tab & 0.684 & 0.573 & 0.108 & 0.773 & 0.535 ± 0.256 & 2.6 ± 0.1 & 18.8 ± 0.3 \\
KNNImputer_5 & KNNImputer & XGBoostRegressor_tuned & 0.550 & 0.623 & 0.468 & 0.429 & 0.517 ± 0.075 & 2.5 ± 0.0 & 5.0 ± 0.2 \\
NoImputer & NoImputer & XGBoostRegressor & 0.543 & 0.636 & 0.246 & 0.636 & 0.515 ± 0.160 & nan ± nan & 0.8 ± 0.0 \\
NoImputer & NoImputer & RandomForestRegressor & 0.548 & 0.624 & 0

In [66]:
latex_df, latex_r2 = generate_metric_table(
    results_list=dict_results_loo_nona,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='r2',
    source="Adjusted",
    csv_filename="../tables/3_training_loonona_r2_adjusted_sorted.csv",
    sort_order="descending"
)
print(latex_r2)

\begin{tabular}{llllllllll}
\toprule
continuous_imputer & ordinal_imputer & model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & mean ± std & imp_time (s) & fit_time (s) \\
\midrule
KNNImputer_5 & KNNImputer & TabNetRegressor_default & 0.292 & 0.267 & 0.236 & 0.312 & 0.277 ± 0.028 & 2.5 ± 0.0 & 12.4 ± 0.7 \\
KNNImputer_5 & KNNImputer & XGBoostRegressor & 0.195 & 0.378 & 0.029 & 0.192 & 0.199 ± 0.123 & 2.5 ± 0.0 & 1.1 ± 0.1 \\
KNNImputer_5 & KNNImputer & RandomForestRegressor & 0.176 & 0.410 & 0.102 & 0.098 & 0.197 ± 0.127 & 2.5 ± 0.0 & 39.8 ± 0.5 \\
KNNImputer_5 & KNNImputer & GatedAdditiveTreeEnsembleConfig_tab & 0.238 & 0.318 & -0.028 & 0.095 & 0.156 ± 0.133 & 2.6 ± 0.1 & 18.8 ± 0.3 \\
NoImputer & NoImputer & XGBoostRegressor & 0.051 & 0.301 & -0.000 & 0.103 & 0.114 ± 0.114 & nan ± nan & 0.8 ± 0.0 \\
NoImputer & NoImputer & RandomForestRegressor & 0.084 & 0.290 & 0.012 & 0.023 & 0.102 ± 0.112 & nan ± nan & 32.5 ± 0.3 \\
KNNImputer_5 & KNNImputer & MultiTaskElasticNet_tuned & 0.100 & 0.3

In [67]:
latex_df, latex_mae = generate_metric_table(
    results_list=dict_results_loo_nona,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='mae',
    source="Adjusted",
    csv_filename="../tables/3_training_loonona_mae_adjusted_sorted.csv",
    sort_order="ascending"
)
print(latex_mae)

\begin{tabular}{llllllllll}
\toprule
continuous_imputer & ordinal_imputer & model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & mean ± std & imp_time (s) & fit_time (s) \\
\midrule
KNNImputer_5 & KNNImputer & TabNetRegressor_default & 0.648 & 0.624 & 0.550 & 0.592 & 0.603 ± 0.037 & 2.5 ± 0.0 & 12.4 ± 0.7 \\
KNNImputer_5 & KNNImputer & RandomForestRegressor & 0.667 & 0.608 & 0.619 & 0.636 & 0.633 ± 0.022 & 2.5 ± 0.0 & 39.8 ± 0.5 \\
KNNImputer_5 & KNNImputer & XGBoostRegressor & 0.653 & 0.639 & 0.626 & 0.617 & 0.633 ± 0.013 & 2.5 ± 0.0 & 1.1 ± 0.1 \\
KNNImputer_5 & KNNImputer & LinearRegression & 0.692 & 0.539 & 0.633 & 0.753 & 0.654 ± 0.079 & 2.7 ± 0.1 & 0.2 ± 0.1 \\
KNNImputer_5 & KNNImputer & GatedAdditiveTreeEnsembleConfig_tab & 0.644 & 0.673 & 0.647 & 0.670 & 0.658 ± 0.013 & 2.6 ± 0.1 & 18.8 ± 0.3 \\
KNNImputer_5 & KNNImputer & MultiTaskLasso_tuned & 0.692 & 0.554 & 0.642 & 0.748 & 0.659 ± 0.071 & 2.6 ± 0.0 & 1.1 ± 0.0 \\
NoImputer & NoImputer & XGBoostRegressor & 0.736 & 0.702 & 0.62

In [68]:
latex_df, latex_mse = generate_metric_table(
    results_list=dict_results_loo_nona,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='mse',
    source="Adjusted",
    csv_filename="../tables/3_training_loonona_mse_adjusted_sorted.csv",
    sort_order="ascending"
)
print(latex_mse)

\begin{tabular}{llllllllll}
\toprule
continuous_imputer & ordinal_imputer & model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & mean ± std & imp_time (s) & fit_time (s) \\
\midrule
KNNImputer_5 & KNNImputer & TabNetRegressor_default & 0.702 & 0.647 & 0.367 & 0.566 & 0.570 ± 0.127 & 2.5 ± 0.0 & 12.4 ± 0.7 \\
KNNImputer_5 & KNNImputer & XGBoostRegressor & 0.797 & 0.549 & 0.466 & 0.665 & 0.619 ± 0.125 & 2.5 ± 0.0 & 1.1 ± 0.1 \\
KNNImputer_5 & KNNImputer & RandomForestRegressor & 0.816 & 0.520 & 0.431 & 0.742 & 0.627 ± 0.157 & 2.5 ± 0.0 & 39.8 ± 0.5 \\
KNNImputer_5 & KNNImputer & GatedAdditiveTreeEnsembleConfig_tab & 0.755 & 0.602 & 0.494 & 0.744 & 0.649 ± 0.108 & 2.6 ± 0.1 & 18.8 ± 0.3 \\
NoImputer & NoImputer & XGBoostRegressor & 0.940 & 0.617 & 0.480 & 0.738 & 0.694 ± 0.169 & nan ± nan & 0.8 ± 0.0 \\
KNNImputer_5 & KNNImputer & MultiTaskElasticNet_tuned & 0.892 & 0.546 & 0.523 & 0.815 & 0.694 ± 0.162 & 2.7 ± 0.1 & 1.4 ± 0.1 \\
NoImputer & NoImputer & RandomForestRegressor & 0.907 & 0.626 